# Probing Task Approach
by Lucrezia Labardi

## import libraries and load drive and folder

In [ ]:
import tokenizers
import transformers
from transformers import BertTokenizer, BertForMaskedLM
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from transformers.pipelines import pipeline
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/MyDrive/Tesi-magistrale')

Mounted at /content/drive


## Select data

In [ ]:
train_df = pd.read_csv("Probing/probing_train.csv")
val_df = pd.read_csv("Probing/probing_test.csv")

In [ ]:
train_dict = train_df.to_dict(orient='records')
val_dict = val_df.to_dict(orient='records')

In [ ]:
train_dict[0]

{'id': 'isdt_tut-689',
 'sent': "Il diritto dell'usufruttuario non si estende al tesoro che si scopra durante l'usufrutto, salve le ragioni che gli possono competere come ritrovatore (932).",
 'category': 8,
 'n_tokens': 31,
 'char_per_tok': 4.81481481481482,
 'upos_dist_DET': 16.1290322580645,
 'upos_dist_ADV': 3.2258064516129,
 'upos_dist_PUNCT': 12.9032258064516,
 'upos_dist_NUM': 3.2258064516129,
 'upos_dist_PRON': 16.1290322580645,
 'upos_dist_ADP': 12.9032258064516,
 'upos_dist_PROPN': 0.0,
 'upos_dist_ADJ': 3.2258064516129,
 'upos_dist_VERB': 9.67741935483871,
 'upos_dist_NOUN': 19.3548387096774,
 'upos_dist_CCONJ': 0.0,
 'upos_dist_AUX': 3.2258064516129,
 'avg_links_len': 2.42307692307692,
 'max_links_len': 11,
 'avg_max_depth': 5,
 'dep_dist_obj': 0.0,
 'dep_dist_nsubj': 12.9032258064516,
 'subj_pre': 75.0,
 'subj_post': 25.0,
 'n_prepositional_chains': 1,
 'avg_prepositional_chain_len': 1.0,
 'avg_subordinate_chain_len': 1.0,
 'subordinate_proposition_dist': 66.6666666666667,

## Function definition

In [ ]:
def feature_extraction(samples, model_name):
  "Function to get sentence embeddings from layer 1 to 8"
    first_layer = 1
    last_layer = 8
    #load tokenizer
    tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
    #load model
    model = BertForMaskedLM.from_pretrained(model_name)
    for sample in tqdm(samples, desc="Estrazione features", unit="sample"):
        encoded_sen = tokenizer(sample["sent"], padding=True, truncation=True, max_length=128, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_sen, output_hidden_states=True)
            hidden_states = model_output.hidden_states
            #only select [CLS] embedding
            for layer in range(first_layer, last_layer+1):
                layer_output = torch.squeeze(hidden_states[layer])
                cls_embedding = layer_output[0, :].cpu().detach().numpy()
                sample[f'layer_{layer}'] = {'cls_embedding': cls_embedding}
    return samples

In [ ]:
#funzione per ottenere features e lables
def get_features_lables(samples, feature, layer):
  "Function to get features and labels"
    X = []
    y = []
    for sample in samples:
        emb = sample[layer]["cls_embedding"]
        label =  sample[feature]
        X.append(emb)
        y.append(label)
    return X, y

In [ ]:
def train_eval(train_set, val_set, feature, layer):
  "Function to train and evaluate the probe"
    #define scaler and apply to array of features (embeddings)
    scaler = MinMaxScaler()
    X_train, y_train = get_features_lables(train_set, feature, layer)
    X_val, y_val = get_features_lables(val_set, feature, layer)
    X_train = np.array(X_train)
    X_val = np.array(X_val)
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_val = scaler.transform(X_val)
    #train Ridge regressor and get prediction
    clf = sklearn.linear_model.Ridge(alpha=1.0)
    clf.fit(scaled_X_train, y_train)
    y_pred = clf.predict(scaled_X_val)
    #return predictions
    return y_pred

## Parameter definition and do probing

In [ ]:
#parameters definition
checkpoints = [2, 32, 512, 8192, 15449]
ling_features = ["n_tokens",  "char_per_tok", "upos_dist_DET", "upos_dist_ADV", "upos_dist_PUNCT", "upos_dist_NUM", "upos_dist_PRON", "upos_dist_ADP", "upos_dist_PROPN","upos_dist_ADJ","upos_dist_VERB","upos_dist_NOUN", "upos_dist_CCONJ", "upos_dist_AUX", "avg_links_len", "max_links_len", "avg_max_depth", "dep_dist_obj", "dep_dist_nsubj", "subj_pre", "subj_post", "n_prepositional_chains", "avg_prepositional_chain_len", "avg_subordinate_chain_len", "subordinate_proposition_dist", "avg_verb_edges"]
training_id = "Anti-Gulpease" #ReadIt, Anti-ReadIt, Gulpease, Anti-Gulpease, Random1, Random2, Random3, Random4, Random5

In [ ]:
def probing_checkpoints(checkpoints, training_id, train_dict, val_dict, ling_features):
  "Function to do the probing using all the other functions"
    first_layer = 1
    last_layer = 8
    results = pd.DataFrame()
    #for each checkpoint of the model
    for n_step in checkpoints:
        checkpoint_name = f'checkpoint-{n_step}'
        checkpoint = n_step
        model_name = f"Pre-training/model_folder/checkpoint-{n_step}"
        print(f"Starting probing for checkpoint {n_step}")
        print("Extracting training features (embeddings)...")
        train_samples = feature_extraction(train_dict, model_name)
        print("Extracting validation features (embeddings)...")
        val_samples = feature_extraction(val_dict, model_name)
        # for each linguistic feature in that checkpoint
        for ling_feature in ling_features:
            print(f'Training the model on feature: {ling_feature}')
            #for each layer
            for layer in range(first_layer, last_layer+1):
                layer_result = train_eval(train_samples, val_samples, ling_feature, f'layer_{layer}')
                #save in a dictionary
                row = {"model": training_id, "step": checkpoint, "ling_feature": ling_feature, "layer": layer, "preds": layer_result}
                results = results._append(row, ignore_index = True)
    return results


In [ ]:
final_results = probing_checkpoints(checkpoints, training_id, train_dict, val_dict, ling_features)

Inizio probing per il checkpoint 2048
Estrazione delle feature di training...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

...

Addestramento del modello sulla feature linguistica: n_tokens
Addestramento del modello sulla feature linguistica: char_per_tok
Addestramento del modello sulla feature linguistica: upos_dist_DET
Addestramento del modello sulla feature linguistica: upos_dist_ADV
Addestramento del modello sulla feature linguistica: upos_dist_PUNCT
Addestramento del modello sulla feature linguistica: upos_dist_NUM
Addestramento del modello sulla feature linguistica: upos_dist_PRON
Addestramento del modello sulla feature linguistica: upos_dist_ADP
Addestramento del modello sulla feature linguistica: upos_dist_PROPN
Addestramento del modello sulla feature linguistica: upos_dist_ADJ
Addestramento del modello sulla feature linguistica: upos_dist_VERB
Addestramento del modello sulla feature linguistica: upos_dist_NOUN
Addestramento del modello sulla feature linguistica: upos_dist_CCONJ
Addestramento del modello sulla feature linguistica: upos_dist_AUX
Addestramento del modello sulla feature linguistica: avg_li

Estrazione features: 100%|██████████| 10000/10000 [13:53<00:00, 12.00sample/s]


Estrazione delle features di validation...


Estrazione features: 100%|██████████| 5000/5000 [07:03<00:00, 11.81sample/s]


Addestramento del modello sulla feature linguistica: n_tokens
Addestramento del modello sulla feature linguistica: char_per_tok
Addestramento del modello sulla feature linguistica: upos_dist_DET
Addestramento del modello sulla feature linguistica: upos_dist_ADV
Addestramento del modello sulla feature linguistica: upos_dist_PUNCT
Addestramento del modello sulla feature linguistica: upos_dist_NUM
Addestramento del modello sulla feature linguistica: upos_dist_PRON
Addestramento del modello sulla feature linguistica: upos_dist_ADP
Addestramento del modello sulla feature linguistica: upos_dist_PROPN
Addestramento del modello sulla feature linguistica: upos_dist_ADJ
Addestramento del modello sulla feature linguistica: upos_dist_VERB
Addestramento del modello sulla feature linguistica: upos_dist_NOUN
Addestramento del modello sulla feature linguistica: upos_dist_CCONJ
Addestramento del modello sulla feature linguistica: upos_dist_AUX
Addestramento del modello sulla feature linguistica: avg_li

## Visualise and save results

In [ ]:
final_results.head()

,model,step,ling_feature,layer,preds
0,Anti-Gulpease,2048,n_tokens,1,"[6.894903, 26.457176, 15.04243, 25.056557, 22...."
1,Anti-Gulpease,2048,n_tokens,2,"[9.544374, 24.546638, 10.99552, 27.219418, 19...."
2,Anti-Gulpease,2048,n_tokens,3,"[9.022346, 27.199722, 10.849664, 24.095848, 20..."
3,Anti-Gulpease,2048,n_tokens,4,"[12.27919, 28.478298, 6.116555, 25.934109, 19...."
4,Anti-Gulpease,2048,n_tokens,5,"[8.177292, 29.683025, 8.726753, 28.390915, 18...."


In [ ]:
# save in a json file
import json
result = final_results.to_json(f'predictions_{training_id}.json', orient="columns")